# Functions

This is a Jupyter Notebook to explain the functions used in the local / global explainability master thesis.

Author: Roman Zogg  
Email: od21r2z@leeds.ac.uk 
Written: 2023

## Importing libraries

The main standardlibraries are pandas, numpy, matplotlib and seaborn.  
For the modelling Scikit Learn and XGBoost are used.  
For SHAP and LIME the repsective libraries

In [1]:
# Import the main libraries for the functions below
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.feature_selection import f_regression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, BayesianRidge
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from warnings import simplefilter, filterwarnings
from collections import defaultdict
import matplotlib as mpl
import squarify
import re
import shap
import lime
from collections import defaultdict
from collections import OrderedDict

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Function: `modelSelection`
The function `modelSelection` evaluates various classification models based on recall, precision, and F1-score using stratified cross-validation. It visualizes the results using box plots for each metric.

The code for this function has been inspired by:  
Lukas Frei (https://towardsdatascience.com/detecting-credit-card-fraud-using-machine-learning-a3d83423d3b8)



#### Parameters:

- `X` : array-like or pd.DataFrame, shape (n_samples, n_features)
    - Training data.
<br> <br>
- `y` : array-like or pd.Series, shape (n_samples)
    - Target labels.
  <br> <br>
- `n_split` : int, optional, default=5
    - Determines the number of folds in stratified cross-validation.
  <br> <br>  
- `random_state` : int or None, optional, default=42
    - Random seed for reproducibility. 
    - None for random initialization.
<br> <br>
#### Outputs:

- Visualizes box plots showing the performance of each model on recall, precision, and F1-score.
- Saves the visualized plots as "model_selection_plot.png" at a high resolution (1200 dpi).

#### Description:

1. **Model Definitions**: The function evaluates the performance of several classification models. These models include:
   - Logistic Regression
   - Decision Tree Classifier
   - Random Forest Classifier
   - Neural Network (MLP Classifier)
   - Gaussian Naive Bayes
   - XGBoost Classifier
   - Support Vector Classifier
<br> <br>
2. **Cross-Validation**: The function employs stratified cross-validation to account for imbalances in the target variable, which is important in classification tasks. The number of splits/folds in cross-validation is set by `n_split`.

3. **Metric Evaluation**:
    - Recall, precision, and F1-score for each model are computed.
    - Results of these evaluations are stored in respective lists for visualization.
<br><br>
4. **Visualization**:
    - Utilizes Matplotlib to visualize the performance metrics in the form of box plots.
    - Generates separate box plots for recall, precision, and F1-score of the evaluated models.
    - Customizes the appearance (e.g., font family, font size) for improved readability.
<br><br>
5. **Saving Visualization**:
    - The created visualization is saved as an image with the filename "model_selection_plot.png".
    - The image is saved with high resolution (1200 dpi) to preserve details.



In [ ]:
# Model Selection

def modelSelection(X, y, n_split = 5, random_state = 42):
    models = []
    
    # Defining the models
    models.append(('LogReg', LogisticRegression(class_weight = 'balanced', max_iter = 200, random_state = random_state)))
    models.append(('Tree', DecisionTreeClassifier(random_state = random_state)))
    models.append(('RandForest', RandomForestClassifier(class_weight = 'balanced', random_state = random_state)))
    models.append(('NeuralNet', MLPClassifier(solver = 'adam', hidden_layer_sizes=(120, 60, 10), max_iter=100, random_state = random_state)))
    models.append(('NaiveBayes', GaussianNB()))
    models.append(('XGB', XGBClassifier(random_state = random_state)))
    #models.append(('KNearN', KNeighborsClassifier()))
    models.append(('SuppVec', SVC(class_weight = 'balanced', random_state = random_state)))
 
    
    # Runing the Cross validation in Recall
    recallResult = []
    recallModelName = []
    
    for name, model in models:
        print(f"Processing Recall for model: {name}")  # Debug print
        kFold = StratifiedKFold(n_splits = n_split, shuffle = True, random_state = random_state)
        cvResults = cross_val_score(model, X, y, cv=kFold, scoring='recall')
        recallResult.append(cvResults)
        recallModelName.append(name)
        
    # Runing the Cross validation in F1
    f1Result = []
    f1ModelName = []
        
    for name, model in models:
        print(f"Processing F1 for model: {name}")  # Debug print
        kFold = StratifiedKFold(n_splits = n_split, shuffle = True, random_state = random_state)
        cvResults = cross_val_score(model, X, y, cv=kFold, scoring='f1')
        f1Result.append(cvResults)
        f1ModelName.append(name)    
     
    # Runing the Cross validation in Precision
    precisionResult = []
    precisionModelName = []
    
    for name, model in models:
        print(f"Processing Precision for model: {name}")  # Debug print
        kFold = StratifiedKFold(n_splits = n_split, shuffle = True, random_state = random_state)
        cvResults = cross_val_score(model, X, y, cv=kFold, scoring='precision')
        precisionResult.append(cvResults)
        precisionModelName.append(name) 
    
    # Setting the font
    mpl.rcParams['font.family'] = 'Times New Roman'
    font_size = 13  # Change this value to adjust the font size
    
    # Plotting the results    
    fig, axes = plt.subplots(3, 1, figsize=(11.8, 10), sharey=True)
    
    axes[0].boxplot(recallResult) 
    axes[0].set_title('Recall of Classification Algorithms', fontsize = font_size+2)
    axes[0].set_ylabel('Recall', fontsize = font_size+1)
    axes[0].set_xticklabels([])
    
    axes[1].boxplot(precisionResult)
    axes[1].set_title('Precision of Classification Algorithms', fontsize = font_size+2)
    axes[1].set_ylabel('Precision', fontsize = font_size+1)
    axes[1].set_xticklabels([])
    
    axes[2].boxplot(f1Result)
    axes[2].set_title('F1 of Classification Algorithms', fontsize = font_size+2)
    axes[2].set_xlabel('Algorithm', fontsize = font_size+1)
    axes[2].set_ylabel('F1', fontsize = font_size+1)
    axes[2].set_xticklabels(f1ModelName, fontsize = font_size+1)
    
    fig.tight_layout(pad=3.0)
    
    
    # Save the figure with 800 dpi
    fig.savefig("model_selection_plot.png", dpi=1200, bbox_inches='tight')
    
    # Show the plots
    plt.show()

## Function: `parameterTuning`
Performs hyperparameter tuning using grid search on a given estimator using stratified cross-validation. The function also splits the data into training and test sets before conducting the grid search.

#### Parameters:

- `X` : array-like or pd.DataFrame, shape (n_samples, n_features)
    - Training data.
  <br> <br>
- `y` : array-like or pd.Series, shape (n_samples)
    - Target labels.
<br> <br>
- `paramGrid` : dict or list of dictionaries
    - Dictionary with parameter names (string) as keys and lists of parameter settings to try as values, or a list of such dictionaries, in which case the grids spanned by each dictionary in the list are explored.
  <br> <br>
- `estimator` : estimator object
    - The estimator for which the hyperparameters are being tuned.
  <br> <br>  
- `split` : float, optional, default=0.75
    - Represents the proportion of the data to include in the training set. 
    - Should be between 0.0 and 1.0.
  <br> <br>
- `scoring` : string, callable, dict or None, optional, default='f1'
    - Scoring metric to evaluate the predictions on the test set. For the provided function, it's set to default as 'f1'.
    - If single string: Evaluates the predictions on the test set using the provided single metric. Common options are 'precision', 'recall', 'f1', etc.
    - If None, the estimator’s default scorer is used.
    - For more details on valid scoring strings or callables, refer to Scikit-learn's [model evaluation documentation](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).
   <br> <br> 
- `n_split` : int, optional, default=3
    - Determines the number of folds in stratified cross-validation.
<br> <br>
- `random_state` : int or None, optional, default=42
    - Random seed for reproducibility. 
    - None for random initialization.
<br> <br>
#### Returns:

- `bestParams` : dict
    - Dictionary containing the best hyperparameters found by the grid search.
<br> <br>
#### Description:

1. **Data Splitting**: 
    - The function first splits the provided data (`X`, `y`) into training and test sets using the `train_test_split` method. 
    - It ensures that the split is stratified to maintain the proportion of target classes.
  <br> <br>
2. **Cross-Validation Setup**: 
    - A stratified cross-validation object is created to be used in the grid search. 
    - Stratified cross-validation ensures that each fold has the same proportion of observations with a given target value as the complete dataset.
<br> <br>
3. **Grid Search Setup**:
    - A `GridSearchCV` object is initialized with the provided estimator, parameter grid, cross-validation object, and scoring metric.
<br> <br>
4. **Conducting Grid Search**:
    - The grid search is performed on the training data. The search explores all combinations of parameters provided in the `paramGrid`.
   <br> <br> 
5. **Result Retrieval**:
    - After the grid search is complete, the best hyperparameters are extracted and printed.
    - The best hyperparameters are returned by the function.


In [ ]:
# writing a function for Hyperparameter tuning

def parameterTuning (X, y, paramGrid, estimator, split = 0.75, scoring = 'f1', n_split = 3, random_state = 42):
    
    # Split the data set into test and train
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify = y, 
        train_size = split, 
        random_state = random_state)
    
    # Strafified Cross Validator
    cv = StratifiedKFold(
        n_splits = n_split,
        shuffle = True,
        random_state = random_state
    )
    
    # Creating Grid Search object
    gridSearch = GridSearchCV(
        estimator = estimator,
        param_grid = paramGrid,
        cv = cv,
        scoring = scoring
    )
    
    # Conduct search
    gridResult = gridSearch.fit(X_train, y_train)
    
    # Get the best parameters
    bestParams = gridResult.best_params_
    
    print(bestParams)
    
    return bestParams


## Function: `confusionMatrix`

The `confusionMatrix` function evaluates a given machine learning model's performance on a dataset by plotting its confusion matrix and printing some performance metrics, including Recall, Precision, and F1 Score.

#### Parameters:

- `X` : array-like, shape (n_samples, n_features)
    - Feature matrix. The samples and set of features to be used for testing and training the model.
<br><br>
- `y` : array-like, shape (n_samples,)
    - True labels for the samples in `X`.
<br><br>
- `model` : Scikit-learn classifier instance
    - The machine learning model or classifier which needs to be evaluated. This should have `fit` and `predict` methods, typical of Scikit-learn models.
<br><br>
- `split` : float, optional, default=0.75
    - Determines the proportion of the data to be used for training. The remainder will be used for testing. For instance, if split is 0.75, it means 75% of the data is used for training and the rest for testing.
<br><br>
- `random_state` : int, optional, default=42
    - Determines the randomness of the train-test data splitting. Pass an int for reproducible output across multiple function calls.
<br><br>
#### Outputs:

No explicit return. The function plots the confusion matrix for the classifier on the given data. It also prints the Recall, Precision, and F1 Score metrics to provide insights into the model's performance.

#### Function Behavior:

1. The function first splits the given data (`X`, `y`) into training and testing sets based on the provided `split` ratio.
2. The model is trained or fitted using the training data.
3. The trained model is then used to predict the labels for the test data.
4. A confusion matrix is plotted using the true labels and the predicted labels for the test data.
5. The Recall, Precision, and F1 Score performance metrics are computed and printed to provide insights into the model's performance.

#### Example:

```python
from sklearn.ensemble import RandomForestClassifier

# Sample data
X, y = load_sample_data()

# Create a random forest classifier
rf_model = RandomForestClassifier()

# Evaluate model using the confusionMatrix function
confusionMatrix(X, y, rf_model)
```

This would plot the confusion matrix for the random forest classifier and print its Recall, Precision, and F1 Score.


In [ ]:
# Function for the confusion matrix

def confusionMatrix (X, y, model, split = 0.75, random_state = 42):
    
    # Splitting the data into a Testing and tarinign set
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, train_size = split, random_state = random_state)
    
    #fit the training data to the algorithm
    model.fit(X_train, y_train)
    
    # Run the algorithm on the test data
    y_pred = model.predict(X_test)
    
    # Prepare and plot the confusions matrix
    
    plot_confusion_matrix(model, X_test, y_test, cmap=plt.cm.Blues)
    print('\033[1m'+'Performance Metrics' + '\033[0m' + '\nRecall: {:.2f}% \nPrecision: {:.2f}%\nF1 Score: {:.2f}%'.format(
        recall_score(y_test, y_pred)*100, precision_score(y_test, y_pred)*100, f1_score(y_test, y_pred)*100))
    

## Function: `shapValue` 

Compute and optionally visualize the SHAP (SHapley Additive exPlanations) values for a given machine learning model, which helps in understanding the impact of each feature on the model's predictions.
<br><br>
#### Parameters:

- `X_train` : DataFrame, shape (n_samples, n_features)
    - The training feature matrix. Used for creating the SHAP explainer object.
<br><br>
- `X_test` : DataFrame, shape (n_samples, n_features)
    - The testing feature matrix. SHAP values are computed for this data.
<br><br>
- `model` : Scikit-learn classifier or regressor instance
    - The machine learning model for which SHAP values are computed. Should implement a `predict` or `predict_proba` method.
<br><br>
- `K` : int, optional, default=25
    - The number of instances sampled from `X_train` to approximate the SHAP values.
<br><br>
- `random_state` : int, optional, default=42
    - Seed used by the random number generator during sampling.
<br><br>
- `plotting` : bool, optional, default=False
    - If True, plots the SHAP summary and bar plots.
<br><br>
#### Returns:

- `shap_dict` : dict
    - A dictionary containing features as keys and their mean absolute SHAP values as values. It provides an understanding of the feature importances as determined by SHAP values.

#### Function Behavior:

1. Samples `K` data points from the training data (`X_train`).
2. Prepares the model for SHAP explainer, ensuring compatibility by wrapping the `predict_proba` method.
3. Creates a SHAP explainer object using the sampled data points.
4. Computes the SHAP values for the test data (`X_test`).
5. Constructs a dictionary of feature importances based on the mean absolute SHAP values.
6. If `plotting` is True, visualizes the SHAP values using summary and bar plots.

#### Example:

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import pandas as pd

# Sample data
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Train-test split
X_train, X_test = train_test_split(X, random_state=42)

# Create a random forest classifier and fit
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y)

# Evaluate SHAP values and visualize
shap_dict = shapValue(X_train, X_test, rf_model, plotting=True)

# Print feature importances based on SHAP values
print(shap_dict)
```

This example computes and visualizes SHAP values for a RandomForest classifier on the Iris dataset and then prints the feature importances.


In [ ]:
# Function for the Shap Value

def shapValue(X_train, X_test, model,  K=25, random_state=42, plotting = False):
    # Sample K data points from the training data
    X_train_sample = shap.sample(X_train, K, random_state=random_state)
    
    # Define a wrapped predict_proba function
    def wrappedPredict(data):
       return model.predict_proba(data)

    # Create the explainer object
    try:
        explainer = shap.KernelExplainer(model.predict_proba, X_train_sample)
    except AttributeError:
        explainer = shap.KernelExplainer(wrappedPredict, X_train_sample)
    
    # Compute SHAP values
    shap_values = explainer.shap_values(X_test)
    
    # Create an Explanation object
    explainer = shap.Explanation(values=shap_values[0], 
                                 data=X_test.values, 
                                 feature_names=X_train.columns)
    
    # Calculate the mean absolute SHAP values for each feature
    mean_shap_values = np.abs(shap_values[0]).mean(0)
    
    # Create a DataFrame for feature importance
    feature_importance = pd.DataFrame(list(zip(X_train.columns, mean_shap_values)), columns=['col_name', 'feature_importance_values'])
    feature_importance.sort_values(by=['feature_importance_values'], ascending=False, inplace=True)

    # Create a dictionary to return
    shap_dict = feature_importance.set_index('col_name')['feature_importance_values'].to_dict()
    
    # Plotting
    if plotting:
        shap.summary_plot(shap_values[0], X_test, X_train.columns.tolist())
        shap.plots.bar(explainer, max_display=12)
    
    return shap_dict
    

## Function: `localExplanation` 

Compute a local explanation of a given machine learning model's prediction using LIME (Local Interpretable Model-agnostic Explanations). It gives insight into how each feature contributes to a specific instance's prediction.
<br><br>
#### Parameters:

- `X` : DataFrame, shape (n_samples, n_features)
    - The feature matrix. Used for creating the LIME explainer object.
<br><br>
- `y` : array-like, shape (n_samples,)
    - The target variable. Used for the LIME explainer's initialization.
<br><br>
- `model` : Scikit-learn classifier or regressor instance
    - The machine learning model for which local explanations are computed. It should implement a `predict` or `predict_proba` method.
<br><br>
- `explainInstance` : int, optional, default=0
    - The index of the instance in `X` for which the local explanation is computed.
<br><br>
- `plotting` : bool, optional, default=False
    - If True, displays the LIME explanation plot in a notebook environment.
<br><br>
#### Returns:

- `mean_lime_values` : dict
    - A dictionary containing features as keys and their average LIME values as values for the explained instance.

#### Function Behavior:

1. Initializes a LIME explainer using the provided data (`X` and `y`).
2. Defines a dictionary to store feature effects.
3. Generates local explanations for the specified instance using the provided model.
4. Aggregates feature effects for the explained instance.
5. If `plotting` is True, visualizes the LIME explanations.

#### Example:

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import pandas as pd

# Sample data
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Create a random forest classifier and fit
rf_model = RandomForestClassifier()
rf_model.fit(X, y)

# Generate local explanation for the first instance and visualize
lime_dict = localExplanation(X, y, rf_model, explainInstance=0, plotting=True)

# Print local feature importances based on LIME
print(lime_dict)
```

In this example, a local explanation is generated for the first instance of the Iris dataset using a RandomForest classifier. The LIME visualizations are displayed, and the local feature importances are printed.


In [ ]:
#Function for LIME values

def localExplanation(X, y, model, explainInstance=0, plotting = False):
    numberFeatures = X.shape[1]
    explainer = lime.lime_tabular.LimeTabularExplainer(
        X.values,
        training_labels=y,
        feature_names=X.columns.tolist(),
        class_names=['0', '1'],
        mode='classification'
    )

    feature_names = X.columns.tolist()
    feature_effects = {name: [] for name in feature_names}
    
    indices = [explainInstance]  # Just for the specific row
   
    for i in indices:
        exp = explainer.explain_instance(
            X.iloc[i].values,
            model.predict_proba,
            num_features=numberFeatures
        )
        
        for feature_condition, effect in exp.as_list():
            for feature_name in feature_names:
                if feature_name in feature_condition:
                    feature_effects[feature_name].append(effect)
                    break
                
    mean_lime_values = {k: np.mean(v) if v else 0 for k, v in feature_effects.items()}
    
    #Plotting
    if plotting:
        exp.show_in_notebook()
        print("Feature names from LIME:", lime_values_dict.keys())
    
    return mean_lime_values

## Function:`generateTreeMap` 

Visualize the impact of features on a machine learning model's prediction using a treemap. The size of each feature box in the treemap is determined by its SHAP value, while the color represents its LIME value.
<br><br>
#### Parameters:

- `shap_values_storage` : dict
    - A dictionary storing precomputed SHAP values for different models.
<br><br>
- `X` : DataFrame, shape (n_samples, n_features)
    - The feature matrix.
<br><br>
- `y` : array-like, shape (n_samples,)
    - The target variable.
<br><br>
- `model` : Scikit-learn classifier or regressor instance
    - The machine learning model for which the treemap is generated.
<br><br>
- `modelName` : str or model object
    - Name of the model or the model object itself. Used for fetching SHAP values from the storage.
<br><br>
- `split` : float, optional, default=0.75
    - Fraction of the data to be used for training. The remaining data is used for testing.
<br><br>
- `K` : int, optional, default=25
    - The number of instances sampled from `X_train` to approximate the SHAP values.
<br><br>
- `random_state` : int, optional, default=42
    - Seed used by the random number generator.
<br><br>
- `explainInstance` : int, optional
    - The index of the instance in `X` for which the LIME explanation is computed.
<br><br>
- `sortBy` : {'SHAP', 'LIME'}, optional, default='SHAP'
    - Criteria for sorting features in the treemap.
<br><br>
- `shapPercentageCutoff` : float, optional, default=0
    - Percentage threshold for filtering out features based on their SHAP values.
<br><br>
- `plottingSHAP` : bool, optional, default=False
    - If True, displays a bar plot of mean SHAP values.
<br><br>
- `plottingLIME` : bool, optional, default=False
    - If True, displays LIME's local explanation for a particular instance.
<br><br>
- `legend` : bool, optional, default=False
    - If True, generates a legend matching feature names to numeric values and saves it as a CSV file.
<br><br>
- `plotName` : str, optional, default="treemapPlot.png"
    - Name of the file where the treemap is saved.
<br><br>
#### Returns:

- No return value. The function directly visualizes the treemap and optionally saves it as an image.

#### Function Behavior:

1. The function splits the dataset into training and testing sets.
2. Fetches precomputed SHAP values or computes them if needed.
3. Calculates LIME values for the specified instance.
4. Normalizes LIME values for color mapping and calculates SHAP percentages.
5. Filters, sorts, and organizes features based on the provided criteria.
6. Plots and saves the treemap.

#### Example:

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import pandas as pd

# Sample data
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Create a random forest classifier and fit
rf_model = RandomForestClassifier()
rf_model.fit(X, y)

# Compute SHAP values (assuming shapValue function is defined)
shap_values_storage = {
    'RandomForestClassifier': shapValue(X_train, X_test, rf_model)
}

# Generate treemap visualization
generateTreeMap(shap_values_storage, X, y, rf_model, modelName='RandomForestClassifier', explainInstance=0, plottingSHAP=True, plottingLIME=True, legend=True)
```

In this example, a treemap visualization is generated for a RandomForest classifier trained on the Iris dataset. The visualization showcases feature importances and effects using SHAP and LIME values, respectively.


In [ ]:
def generateTreeMap(X, y, model, split = 0.75, K = 25, random_state = 42, explainInstance=None, 
                    sortBy = 'SHAP', shapPercentageCutoff = 0, plottingSHAP = True, plottingLIME = True):
    
    # Splitting the data into a Testing and tarinign set
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify = y, 
        train_size = split, 
        random_state = random_state)
    
    # Get the absolute mean SHAP values
    shap_values_dict = shapValue(X_train, X_test, model, K, random_state, plotting = plottingSHAP)  # Assuming shapValue returns absolute mean SHAP values
    
    # Get the feature names from the dataset
    featureNames = X_train.columns.tolist()
    
    # Get the LIME values
    lime_values_dict = localExplanation(X, y, model, explainInstance=explainInstance, plotting = plottingLIME)
    
    # Make sure shap_values and lime_values are ordered the same way
    shap_values = [shap_values_dict[feature] for feature in featureNames]
    lime_values = [lime_values_dict[feature] for feature in featureNames]
    
    # Normalize LIME values to fit into a color map
    lime_max_abs = max(max(lime_values), abs(min(lime_values)))
    norm = mcolors.Normalize(-lime_max_abs, lime_max_abs)  # I've set the range from -lime_max_abs to lime_max_abs
    colors = [plt.cm.RdYlGn(norm(value)) for value in lime_values]
    
    # Calculate SHAP value percentages
    total_shap = sum(shap_values)
    shap_percentages = [(shap / total_shap) * 100 for shap in shap_values]
    features_with_percentages = [f'{feature} ({percentage:.2f}%)' for feature, percentage in zip(featureNames, shap_percentages)]
    
    
    # Zip all lists together
    zipped_lists = zip(shap_values, lime_values, featureNames, shap_percentages)

    # Sort by SHAP values (you can also sort by absolute LIME values by changing the key)
    if sortBy == 'SHAP':
        sorted_zipped_lists = sorted(zipped_lists, key=lambda x: x[0], reverse=True)
    elif sortBy == 'LIME':
        sorted_zipped_lists = sorted(zipped_lists, key=lambda x: x[1], reverse=True)
    else:
        raise ValueError("Invalid value for sortBy. Choose 'SHAP' or 'LIME'.")
    
    # Unzip sorted lists
    sorted_shap_values, sorted_lime_values, sorted_feature_names, sorted_shap_percentages = zip(*sorted_zipped_lists)

    # Filter features with SHAP percentage higher than the cutoff
    filtered_indices = [i for i, perc in enumerate(sorted_shap_percentages) if perc >= shapPercentageCutoff]
    filtered_shap_values = [sorted_shap_values[i] for i in filtered_indices]
    filtered_lime_values = [sorted_lime_values[i] for i in filtered_indices]
    filtered_feature_names = [sorted_feature_names[i] for i in filtered_indices]
    filtered_shap_percentages = [sorted_shap_percentages[i] for i in filtered_indices]

    features_with_percentages = [f'{name} ({perc:.2f}%)' for name, perc in zip(filtered_feature_names, filtered_shap_percentages)]

    # Normalize LIME values for the color map
    lime_max_abs = max(max(filtered_lime_values), abs(min(filtered_lime_values)))
    norm = mcolors.Normalize(-lime_max_abs, lime_max_abs)
    filtered_colors = [plt.cm.RdYlGn(norm(value)) for value in filtered_lime_values]
    
    # Create a treemap
    plt.figure(figsize=(20, 20))
    squarify.plot(sizes=filtered_shap_values, label=features_with_percentages, color=filtered_colors, alpha=0.7, edgecolor="k", linewidth=1, pad = 1)
    plt.title("Feature Importance and Effect")
    plt.axis('off')
    
    # Add color bar
    sm = plt.cm.ScalarMappable(cmap=plt.cm.RdYlGn, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ticks=np.linspace(-lime_max_abs, lime_max_abs, 5))
    cbar.set_label('Normalized LIME value')
    
    plt.show()

## Helper Functions

Functions neede in the above functions, but not intended to be for general use.

In [ ]:
#Function to show SHAp value in generateTreeMap
def plot_mean_shap_from_dict(shap_values_dict):
    sorted_items = sorted(shap_values_dict.items(), key=lambda x: x[1], reverse=True)
    features, values = zip(*sorted_items)
    
    plt.figure(figsize=(10, len(features)*0.4))
    plt.barh(features, values, color='royalblue')
    plt.xlabel("Mean SHAP Value")
    plt.title("Feature Importance")
    plt.gca().invert_yaxis()  # to display the most important feature at the top
    plt.savefig('shap_mean_plot.png', dpi=800, bbox_inches='tight')
    plt.show()
    

In [ ]:
# Function to calculate NHHI
def calc_NHHI(shap_dict):
    total = sum(shap_dict.values())
    squared_proportions = [(x / total) ** 2 for x in shap_dict.values()]
    NHHI = sum(squared_proportions)
    return NHHI


In [ ]:
# Function to calculate composite metrics for multiple models and store SHAP values
def calc_composite_and_store_shap(X_train, X_test, y_test, models, metric="f1", alpha=0.5, K=25, random_state=42, T=None):
    NHHIs = []
    NHHIs_dict = {}  # Dictionary to store NHHI values for each model
    composite_metrics = {}
    shap_values_storage = {}
    
    # Calculate the NHHI for each model and store SHAP values
    for model in models:
        shap_dict = shapValue(X_train, X_test, model, K=K, random_state=random_state, plotting=False)
        NHHI = calc_NHHI(shap_dict)
        NHHIs.append(NHHI)
        
        model_name = str(model).split("(")[0]
        shap_values_storage[model_name] = shap_dict
        
        NHHIs_dict[model_name] = NHHI  # Store the NHHI value for the current model
    
    # If T is not provided, calculate the mean NHHI
    if T is None:
        T = np.mean(NHHIs)
    
    # Calculate the composite metric for each model
    for i, model in enumerate(models):
        NHHI = NHHIs[i]
        
        # Get model predictions
        y_pred = model.predict(X_test)
        
        # Get chosen metric (Recall, Precision, F1)
        if metric == "f1":
            chosen_metric = f1_score(y_test, y_pred)
        elif metric == "precision":
            chosen_metric = precision_score(y_test, y_pred)
        elif metric == "recall":
            chosen_metric = recall_score(y_test, y_pred)
        else:
            raise ValueError("Invalid metric choice. Please choose 'f1', 'precision', or 'recall'.")
        
        # Calculate penalty term based on deviation from ideal NHHI (T)
        P = abs(NHHI - T)
        
        # Calculate composite metric
        composite_metric = chosen_metric * (1 - alpha * P)
        model_name = str(model).split("(")[0]
        
        composite_metrics[model_name] = composite_metric
        
        # Sort the composite metrics
        composite_metrics = OrderedDict(sorted(composite_metrics.items(), key=lambda x: x[1], reverse=True))
        
        # Sort the NHHI
        NHHIs_dict =  OrderedDict(sorted(NHHIs_dict.items(), key=lambda x: x[1], reverse=False))
    
    return composite_metrics, shap_values_storage, NHHIs_dict  


## Bibliography

Chen, T. and Guestrin, C., 2016. XGBoost: A Scalable Tree Boosting System. San Francisco, California, Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, pp. 785 - 794.

Harris, C. R. et al., 2020. Array programming with NumPy. Nature, 585(7825), pp. 357 - 362.

Hunter, J. D., 2007. Matplotlib: a 2D graphics environment. Computing in Science & Engineering, 9(3), pp. 90 - 95.

Laserson, U., 2021. Pure Python implementation of the squarify treemap layout algorithm. s.l.:s.n.

Lundberg, S. M. and Lee, S.-I., 2017. A unified Approach to Interpreting Model Predictions. Long Beach, CA, Proceedings of the 31st Conference on Neural Information Processing Systems (NIPS), pp. 4768 - 4777.

McKinney, W., 2010. Data Structures for Statisitcal Computing in Python. s.l., Proceedings of the 9th Python in Science Conference, pp. 56 - 61.

Pedregosa, F. et al., 2011. Scikit-learn: Machoine learning in Python. Journal of Machine Learning Research, Volume 12, pp. 2825 - 2830.

Ribeiro, M. T., Singh, S. and Guestrin, C., 2016. "Why Should I Trust You?" Explaining the Predictions of Any Classifier. San Francisco, California, KDD '16: Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, pp. 1135 - 1144.

Waskom, M. L., 2021. seaborn: statistical data visualization. Journal of Open Source Software, 6(60), p. 3021
